In [0]:
spark.sql("USE CATALOG ons")
spark.sql("USE SCHEMA dados")

In [0]:
%sql
-- desativação após 2000 por tipo de usina
CREATE OR REPLACE TABLE capacidade_geracao_1_1_desativacao_por_tipo_usina AS
SELECT nom_tipousina,
       COUNT(*) AS qtd_usinas,
       round(SUM(val_potenciaefetiva),0) AS soma_potencia_MW
FROM ons.dados.capacidade_geracao
WHERE dat_desativacao IS NOT NULL
  AND YEAR(dat_desativacao) >= 2000
GROUP BY nom_tipousina
ORDER BY qtd_usinas DESC;

-- Consultar os dados para visualização
SELECT * FROM capacidade_geracao_1_1_desativacao_por_tipo_usina;

In [0]:
%sql
-- desativação após 2000 por combustível
CREATE OR REPLACE TABLE capacidade_geracao_1_2_desativacao_por_combustivel AS
SELECT 
  nom_combustivel,
  COUNT(*) AS qtd_usinas,
  ROUND(SUM(val_potenciaefetiva), 0) AS soma_potencia_MW,
  ROUND(
    100 * SUM(val_potenciaefetiva) 
    / SUM(SUM(val_potenciaefetiva)) OVER (),
    1
  ) AS pareto
FROM ons.dados.capacidade_geracao
WHERE dat_desativacao IS NOT NULL
  AND YEAR(dat_desativacao) >= 2000
GROUP BY nom_combustivel
ORDER BY qtd_usinas DESC;

-- Consultar os dados para visualização
select * from capacidade_geracao_1_2_desativacao_por_combustivel;

In [0]:
%sql
-- ativação após 2000 por tipo de usina
CREATE OR REPLACE TABLE capacidade_geracao_2_1_ativacao_por_tipo_usina AS
SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
       nom_tipousina,
       COUNT(*) AS qtd_usinas
FROM ons.dados.capacidade_geracao
WHERE YEAR(dat_entradaoperacao) >= 1990
GROUP BY FLOOR(YEAR(dat_entradaoperacao)/10)*10, nom_tipousina
ORDER BY decada, qtd_usinas DESC;

-- Consultar os dados para visualização
select * from capacidade_geracao_2_1_ativacao_por_tipo_usina

In [0]:
%sql
-- ativação após 2000 por combustível
CREATE OR REPLACE TABLE capacidade_geracao_2_2_ativacao_por_combustivel AS
SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
       nom_combustivel,
       COUNT(*) AS qtd_usinas
FROM ons.dados.capacidade_geracao
WHERE YEAR(dat_entradaoperacao) >= 1990
GROUP BY FLOOR(YEAR(dat_entradaoperacao)/10)*10, nom_combustivel
ORDER BY decada, qtd_usinas DESC;

-- Consultar os dados para visualização
select * from capacidade_geracao_2_2_ativacao_por_combustivel

In [0]:
%sql
-- participação renovaveis e não renovaveis
CREATE OR REPLACE TABLE capacidade_geracao_3_1_participacao_renovaveis AS
SELECT 
  decada,
  renovavel_MW,
  nao_renovavel_MW
FROM (
  SELECT 
    FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
    ROUND(SUM(CASE WHEN nom_combustivel IN ('EÓLICA','HIDRÁULICA','FOTOVOLTAICA','BIOMASSA') THEN val_potenciaefetiva ELSE 0 END), 0) AS renovavel_MW,
    ROUND(SUM(CASE WHEN nom_combustivel NOT IN ('EÓLICA','HIDRÁULICA','FOTOVOLTAICA','BIOMASSA') THEN val_potenciaefetiva ELSE 0 END), 0) AS nao_renovavel_MW
  FROM ons.dados.capacidade_geracao
  GROUP BY FLOOR(YEAR(dat_entradaoperacao)/10)*10
)
ORDER BY decada;

-- Consultar os dados para visualização
select * from capacidade_geracao_3_1_participacao_renovaveis

In [0]:
%sql
-- quantitativo de desativações
CREATE OR REPLACE TABLE capacidade_geracao_4_1_quantitativo_desativacoes AS
SELECT CASE WHEN dat_desativacao IS NULL THEN 'Ativa' ELSE 'Desativada' END AS status,
       COUNT(*) AS qtd_usinas,
       round(AVG(val_potenciaefetiva),1) AS potencia_media_MW,
       round(SUM(val_potenciaefetiva),1) AS potencia_total_MW
FROM ons.dados.capacidade_geracao
GROUP BY CASE WHEN dat_desativacao IS NULL THEN 'Ativa' ELSE 'Desativada' END;

-- Consultar os dados para visualização
select * from capacidade_geracao_4_1_quantitativo_desativacoes


In [0]:
%sql
-- relação idade e desativação
CREATE OR REPLACE TABLE capacidade_geracao_5_1_relacao_idade_desativacao AS
SELECT FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10 AS inicio_faixa_idade,
       CONCAT(FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10, '-', FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10+9) AS faixa_idade,
       COUNT(*) AS total_usinas,
       SUM(CASE WHEN dat_desativacao IS NOT NULL THEN 1 ELSE 0 END) AS desativadas,
       ROUND(SUM(CASE WHEN dat_desativacao IS NOT NULL THEN 1 ELSE 0 END)*100.0/COUNT(*),1) AS perc_desativadas
FROM ons.dados.capacidade_geracao
WHERE dat_entradaoperacao IS NOT NULL
GROUP BY CONCAT(FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10, '-', FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10+9),
         FLOOR(DATEDIFF(CURRENT_DATE, dat_entradaoperacao)/365/10)*10
ORDER BY inicio_faixa_idade;

-- Consultar os dados para visualização
select * from capacidade_geracao_5_1_relacao_idade_desativacao

In [0]:
%sql
-- concentração de agentes
CREATE OR REPLACE TABLE capacidade_geracao_6_1_concentracao_agentes AS
SELECT nom_agenteproprietario,
       COUNT(*) AS qtd_usinas,
       round(SUM(val_potenciaefetiva),0) AS soma_potencia_MW
FROM ons.dados.capacidade_geracao
WHERE YEAR(dat_entradaoperacao) >= YEAR(CURRENT_DATE)-10
GROUP BY nom_agenteproprietario
ORDER BY soma_potencia_MW DESC
LIMIT 10;

-- Consultar os dados para visualização
select * from capacidade_geracao_6_1_concentracao_agentes

In [0]:
%sql
-- distribuição de combustíveis em térmicas ao longo das décadas
CREATE OR REPLACE TABLE capacidade_geracao_7_1_combustiveis_termicas_decadas AS
SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
       nom_combustivel,
       COUNT(*) AS qtd_usinas,
       round(SUM(val_potenciaefetiva),0) AS soma_potencia_MW
FROM ons.dados.capacidade_geracao
WHERE nom_tipousina LIKE '%TÉRMICA%'
GROUP BY FLOOR(YEAR(dat_entradaoperacao)/10)*10, nom_combustivel
ORDER BY decada, soma_potencia_MW DESC;

-- Consultar os dados para visualização
select * from capacidade_geracao_7_1_combustiveis_termicas_decadas

In [0]:
%sql
-- evolução de combustiveis renováveis e térmicas não renováveis ao longo das décadas
CREATE OR REPLACE TABLE capacidade_geracao_8_1_evolucao_renovavel_decadas AS
SELECT decada,
       round(SUM(CASE WHEN nom_tipousina LIKE '%TÉRMICA%' AND nom_combustivel != 'BIOMASSA' THEN val_potenciaefetiva ELSE 0 END),0) AS termica_MW,
       round(SUM(CASE WHEN nom_combustivel IN ('EÓLICA','HIDRÁULICA','FOTOVOLTAICA','BIOMASSA') THEN val_potenciaefetiva ELSE 0 END),0) AS renovavel_MW,
       ROUND(
    100 * 
    SUM(CASE WHEN nom_combustivel IN ('EÓLICA','HIDRÁULICA','FOTOVOLTAICA','BIOMASSA') THEN val_potenciaefetiva ELSE 0 END) /
    (SUM(CASE WHEN nom_tipousina LIKE '%TÉRMICA%' AND nom_combustivel != 'BIOMASSA' THEN val_potenciaefetiva ELSE 0 END) +
     SUM(CASE WHEN nom_combustivel IN ('EÓLICA','HIDRÁULICA','FOTOVOLTAICA','BIOMASSA') THEN val_potenciaefetiva ELSE 0 END)),
    1
  ) AS pareto
FROM (
    SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
           nom_tipousina,
           nom_combustivel,
           val_potenciaefetiva
    FROM ons.dados.capacidade_geracao
) t
GROUP BY decada
ORDER BY decada;

-- Consultar os dados para visualização
select * from capacidade_geracao_8_1_evolucao_renovavel_decadas

In [0]:
%sql
SELECT nom_tipousina,
       YEAR(dat_entradaoperacao) AS ano_operacao
FROM ons.dados.capacidade_geracao
WHERE dat_entradaoperacao IS NOT NULL;

In [0]:
# Run the SQL and get the result as a Spark DataFrame
df = spark.sql("""
SELECT nom_tipousina,
       YEAR(dat_entradaoperacao) AS ano_operacao
FROM ons.dados.capacidade_geracao
WHERE dat_entradaoperacao IS NOT NULL
""")

# For each unique plant type, calculate the correlation
types = [row['nom_tipousina'] for row in df.select('nom_tipousina').distinct().collect()]
corr_results = {}
for t in types:
    # Create a binary column for the type
    df_type = df.withColumn('is_type', (df['nom_tipousina'] == t).cast('int'))
    # Calculate correlation
    corr = df_type.stat.corr('is_type', 'ano_operacao')
    corr_results[t] = corr

# Display results
print(corr_results)

In [0]:
%sql
SELECT decada,
       nom_combustivel,
       SUM(val_potenciaefetiva) AS soma_potencia_MW
FROM (
    SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
           nom_combustivel,
           val_potenciaefetiva
    FROM ons.dados.capacidade_geracao
    WHERE nom_tipousina LIKE '%TÉRMICA%'
) t
GROUP BY decada, nom_combustivel
ORDER BY decada, soma_potencia_MW DESC;


In [0]:
# Load SQL result into Spark DataFrame
df = spark.sql("""
SELECT decada,
       nom_combustivel,
       SUM(val_potenciaefetiva) AS soma_potencia_MW
FROM (
    SELECT FLOOR(YEAR(dat_entradaoperacao)/10)*10 AS decada,
           nom_combustivel,
           val_potenciaefetiva
    FROM ons.dados.capacidade_geracao
    WHERE nom_tipousina LIKE '%TÉRMICA%'
) t
GROUP BY decada, nom_combustivel
ORDER BY decada, soma_potencia_MW DESC
""")

# Convert to pandas for easier manipulation
pdf = df.toPandas()

# Pivot to get fuel sums per decade
sum_fuel_by_decade = pdf.pivot_table(
    index='decada',
    columns='nom_combustivel',
    values='soma_potencia_MW',
    fill_value=0
)

# Shannon index function
import numpy as np
def shannon_index(row):
    proportions = row / row.sum()
    proportions = proportions[proportions > 0]
    return -np.sum(proportions * np.log(proportions))

# Calculate diversity for each decade
diversity = {dec: shannon_index(sum_fuel_by_decade.loc[dec]) for dec in sum_fuel_by_decade.index}

print(diversity)